# BIG BAD DATA FETCH

In [1]:
# Standard Libraries
import requests
import pandas as pd
import numpy as np
import time
import timeit
import datetime
from dateutil.relativedelta import relativedelta
import pytz
import glob
import matplotlib.pyplot as plt
import os

import gc

# Joblib for model persistence
from joblib import dump, load

from pathlib import Path

import random


In [2]:
#API Credentials

# Load credentials from the text file
with open("reqd_files/cred.txt", "r") as file:
    USERNAME = file.readline().strip()  # Read the first line for the username
    PASSWORD = file.readline().strip()  # Read the second line for the password

BASE_URL = "https://api.theracingapi.com"

In [3]:
def fetch_and_process(endpoint, params=None, query_number=None, max_retries=5):
    retry_delay = 1  # Start with a 5 second delay
    for attempt in range(max_retries):
        start_time = timeit.default_timer()
        
        response = requests.get(f"{BASE_URL}{endpoint}", auth=(USERNAME, PASSWORD), params=params)
        elapsed = (timeit.default_timer() - start_time) * 1000  # Convert to milliseconds

        # Print statement for each query
        if query_number and (query_number == 1 or query_number % 10 == 0):
            print(f"Query {query_number} duration: {elapsed:.2f} ms")

        # Handling 503 Service Unavailable
        if response.status_code == 503:
            print(f"Error 503 on attempt {attempt + 1}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            retry_delay *= 2  # Exponential backoff
        else:
            break  # Exit retry loop if response is not 503

    time.sleep(0.69)  # Ensure not to exceed the rate limit of the API -- can go to 0.5

    # Return data if status code is 200; otherwise, return None
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [4]:
# Define the endpoint
ENDPOINT = "/v1/courses"

# Fetch and process the data
courses_data = fetch_and_process(ENDPOINT, query_number=1)

# Check for data and create a DataFrame
if courses_data:
    df_courses = pd.DataFrame(courses_data['courses'])
    df_courses.to_csv('csv_exports/course_names.csv', index=False)
    print("Course data fetched and saved successfully.")
else:
    print("Error fetching data.")

Query 1 duration: 1968.26 ms
Course data fetched and saved successfully.


# Below is the big block of code that pulls the entire history

In [5]:
# import pandas as pd
# import datetime

# # Initialize an empty DataFrame to store the appended data
# df_list_racecards_pro = pd.DataFrame()

# # Define the start date as yesterday and the end date as 6 days from today
# start_date = datetime.datetime.today() - datetime.timedelta(days=1)
# end_date = start_date + datetime.timedelta(days=7)

# # Calculate the number of days between start date and end date
# num_days = (end_date - start_date).days

# # Define the endpoint
# ENDPOINT = "/v1/racecards/pro"

# # Iterate through each day from today until the next 7 days
# for i in range(num_days + 1):  # +1 to include end_date
#     date = (start_date + datetime.timedelta(days=i)).strftime('%Y-%m-%d')
#     print(f"Fetching data for date: {date}")

#     try:
#         data = fetch_and_process(ENDPOINT, params={"date": date}, query_number=i+1)
#     except Exception as e:
#         print(f"An error occurred while fetching data for {date}: {e}")
#         continue

#     if data:
#         # Directly use the racecards data, as we're not exploding the runners
#         temp_df = pd.DataFrame(data['racecards'])
        
#         # Concatenate the new DataFrame with the main DataFrame
#         df_list_racecards_pro = pd.concat([df_list_racecards_pro, temp_df], ignore_index=True)
#     else:
#         print(f"No data returned for date: {date}")

# # Save the DataFrame to a CSV file
# df_list_racecards_pro.to_csv('csv_exports/future_racecards_pro.csv', index=False)
# print("Racecards pro data for the next 7 days have been fetched and saved.")


In [6]:
# import pandas as pd
# from pathlib import Path

# # Define the path to the CSV file
# csv_path = Path('csv_exports/bulk_results.csv')

# # Check if the CSV file exists
# if csv_path.exists():
#     # Read the CSV file
#     existing_data = pd.read_csv(csv_path)
#     print("CSV file loaded successfully.")

#     # Convert the 'date' column to datetime format
#     existing_data['date'] = pd.to_datetime(existing_data['date'])

#     # Find the earliest date
#     earliest_date = existing_data['date'].min()
#     print(f"Earliest date in the CSV: {earliest_date.strftime('%Y-%m-%d')}")

#     # Find the latest date
#     latest_date = existing_data['date'].max()
#     print(f"Latest date in the CSV: {latest_date.strftime('%Y-%m-%d')}")

#     # Set the start date to the day after the latest date
#     start_date = latest_date + pd.Timedelta(days=1)
#     # Set the end date to yesterday's date
#     end_date = pd.to_datetime("today") - pd.Timedelta(days=1)
    
#     print(f"Start date: {start_date.strftime('%Y-%m-%d')}")
#     print(f"End date: {end_date.strftime('%Y-%m-%d')}")
# else:
#     print("CSV file does not exist.")
    
#     # Set the start date to a default date or a designated start date
#     default_start_date = pd.to_datetime("2015-01-01")
    
#     # You can designate your own start date here
#     designated_start_date = input("Enter a start date (YYYY-MM-DD) or press Enter to use default (2015-01-01): ")
    
#     if designated_start_date:
#         try:
#             start_date = pd.to_datetime(designated_start_date)
#             print(f"Using designated start date: {start_date.strftime('%Y-%m-%d')}")
#         except ValueError:
#             print("Invalid date format. Using default start date.")
#             start_date = default_start_date
#             print(f"Default start date: {start_date.strftime('%Y-%m-%d')}")
#     else:
#         start_date = default_start_date
#         print(f"Default start date: {start_date.strftime('%Y-%m-%d')}")

#     # Set the end date to yesterday's date
#     end_date = pd.to_datetime("today") - pd.Timedelta(days=1)
    
#     print(f"End date: {end_date.strftime('%Y-%m-%d')}")


## This one below is to start again .. just play with the start and end date

In [7]:

# # Define the endpoint
# ENDPOINT = "/v1/results"

# # Generating a list of dates
# start_date = datetime.datetime(2010, 1, 1)
# end_date = datetime.datetime(2010, 1, 15)
# date_range = pd.date_range(start_date, end_date)

# # API Call for Each Day
# all_data = []
# query_number = 1
# error_dates = []  # List to store dates with errors

# for single_date in date_range:
#     formatted_date = single_date.strftime("%Y-%m-%d")
#     print(f"Querying data for date: {formatted_date}")  # Print the date being queried
#     params = {
#         "start_date": formatted_date,
#         "end_date": formatted_date,
#         "limit": 50,
#         "skip": 0,
#     }
    
#     data_found = False  # Flag to check if data was found for the date

#     while True:
#         try:
#             # Fetch and process the data
#             data = fetch_and_process(ENDPOINT, params=params, query_number=query_number)
#             query_number += 1
#             if not data or not data['results']:
#                 break
#             data_found = True  # Set flag to True as data was found
#             all_data.extend(data['results'])
#             params['skip'] += 50  # Increment the skip parameter for the next iteration
#         except Exception as e:
#             print(f"Error occurred on {formatted_date}: {e}")
#             error_dates.append(formatted_date)  # Add the date to the error list
#             break  # Break the inner loop if an error occurs

#     if data_found:
#         print(f"Data found for date: {formatted_date}")
#     else:
#         print(f"No data found for date: {formatted_date}")

# # Assuming all_data is a list of dictionaries with the new data
# try:
#     if all_data:  # Check if there is any new data to append
#         # Convert the list of dictionaries to a DataFrame
#         bulk_results = pd.DataFrame(all_data)
#         if not bulk_results.empty and 'runners' in bulk_results.columns:
#             bulk_results = bulk_results.explode('runners')
#             runners_data = bulk_results['runners'].apply(pd.Series)
#             bulk_results = pd.concat([bulk_results.drop(columns='runners'), runners_data], axis=1)

#         # Define the path to the CSV file
#         csv_path = Path('csv_exports/bulk_results.csv')

#         # Check if the CSV file exists to determine if we need to write headers
#         file_exists = csv_path.exists()

#         # Export to CSV, appending if file exists, including header if file does not exist
#         bulk_results.to_csv(csv_path, mode='a', header=not file_exists, index=False)
#         print("Data appended to CSV successfully.")
# except Exception as e:
#     print(f"Error during data processing or export: {e}")



## This one below creates a sample csv from the above

In [8]:
# # Configuration for sampling
# BLOCK_SIZE = 7      # Number of rows in each block
# NUM_BLOCKS = 10      # Number of blocks to sample

# # Read the main CSV file
# csv_path = Path('csv_exports/bulk_results.csv')
# df = pd.read_csv(csv_path)

# # Calculate how many complete blocks we can have
# total_blocks = len(df) // BLOCK_SIZE

# # Ensure we're not trying to sample more blocks than exist
# if NUM_BLOCKS > total_blocks:
#     print(f"Warning: Requested {NUM_BLOCKS} blocks but only {total_blocks} exist. Using maximum available.")
#     NUM_BLOCKS = total_blocks

# # Generate random block starting points
# # This randomly picks NUM_BLOCKS numbers from range(total_blocks)
# # Each number represents the start of a block
# print("Block starting positions chosen:")
# random_blocks = random.sample(range(total_blocks), NUM_BLOCKS)
# for i, block_start in enumerate(sorted(random_blocks)):
#     print(f"Block {i+1} starts at row {block_start * BLOCK_SIZE}")

# # Initialize list to store selected rows
# selected_rows = []

# # Get blocks for each randomly selected starting point
# for block_start in random_blocks:
#     start_idx = block_start * BLOCK_SIZE
#     end_idx = start_idx + BLOCK_SIZE
#     selected_rows.extend(list(range(start_idx, end_idx)))

# # Create new DataFrame with selected rows
# example_df = df.iloc[selected_rows].copy()

# # Save to new CSV file with explicit header
# example_df.to_csv('csv_exports/example_csv.csv', index=False, header=True)

# print(f"\nCreated example CSV with {len(example_df)} rows")
# print(f"({NUM_BLOCKS} blocks of {BLOCK_SIZE} rows each)")
# print(f"Original CSV had {len(df)} rows")
# print("Column headers have been included in the example CSV")

## This below is to find unseen date and append, rather than start again

In [9]:
# Function to generate a list of missing dates
def generate_missing_dates(start_date, end_date, existing_dates):
    date_range = pd.date_range(start=start_date, end=end_date)
    missing_dates = date_range.difference(existing_dates)
    return missing_dates

# Define the endpoint
ENDPOINT = "/v1/results"

# Function to query data via API for a date range with regular saving
def query_data(date_range, query_number=1, save_frequency=2500, csv_path=None):
    all_data = []
    error_dates = []
    entries_since_save = 0
    
    for single_date in date_range:
        formatted_date = single_date.strftime("%Y-%m-%d")
        print(f"Querying data for date: {formatted_date}")
        params = {
            "start_date": formatted_date,
            "end_date": formatted_date,
            "limit": 50,
            "skip": 0,
        }

        data_found = False

        while True:
            try:
                # Fetch and process the data
                data = fetch_and_process(ENDPOINT, params=params, query_number=query_number)
                query_number += 1
                
                if not data or not data['results']:
                    break
                    
                data_found = True
                
                # Add new results and track how many were added
                new_entries = data['results']
                all_data.extend(new_entries)
                entries_since_save += len(new_entries)
                
                # Save progress if we've accumulated enough new entries
                if csv_path and entries_since_save >= save_frequency:
                    save_progress_to_csv(all_data[-entries_since_save:], csv_path)  # Only save the new entries
                    print(f"Progress saved: {entries_since_save} new entries written to CSV")
                    entries_since_save = 0  # Reset counter after saving
                
                params['skip'] += 50  # Increment the skip parameter for the next iteration
                
            except Exception as e:
                print(f"Error occurred on {formatted_date}: {e}")
                error_dates.append(formatted_date)
                break

        if data_found:
            print(f"Data found for date: {formatted_date}")
        else:
            print(f"No data found for date: {formatted_date}")

    # Save any remaining data that didn't reach the save frequency threshold
    if csv_path and entries_since_save > 0:
        save_progress_to_csv(all_data[-entries_since_save:], csv_path)  # Only save the new entries
        print(f"Final save: {entries_since_save} remaining entries written to CSV")

    return all_data, error_dates

# Function to save progress to CSV
def save_progress_to_csv(data_to_save, csv_path):
    # Process the accumulated data
    if not data_to_save:
        print("No data to save")
        return
        
    bulk_results = pd.DataFrame(data_to_save)
    
    if 'runners' in bulk_results.columns:
        bulk_results = bulk_results.explode('runners')
        runners_data = bulk_results['runners'].apply(pd.Series)
        bulk_results = pd.concat([bulk_results.drop(columns='runners'), runners_data], axis=1)

    # Check if file exists to determine if we need headers
    file_exists = Path(csv_path).exists()
    
    # Export to CSV, appending data
    bulk_results.to_csv(csv_path, mode='a', header=not file_exists, index=False)

# Define the path to the CSV file
csv_path = Path('csv_exports/bulk_results.csv')

# Define the default earliest date
default_earliest_date = pd.to_datetime("2015-01-01")

# Check if the CSV file exists
if csv_path.exists():
    # Read the CSV file
    existing_data = pd.read_csv(csv_path)
    print("CSV file loaded successfully.")

    # Convert the 'date' column to datetime format
    existing_data['date'] = pd.to_datetime(existing_data['date'])

    # Find the earliest and latest dates in the CSV file
    earliest_date = existing_data['date'].min()
    latest_date = existing_data['date'].max()

    print(f"Earliest date in the CSV: {earliest_date.strftime('%Y-%m-%d')}")
    print(f"Latest date in the CSV: {latest_date.strftime('%Y-%m-%d')}")

    # Generate missing dates
    missing_before = generate_missing_dates(default_earliest_date, earliest_date - pd.Timedelta(days=1), existing_data['date'])
    start_date = latest_date + pd.Timedelta(days=1)
    end_date = pd.to_datetime("today") - pd.Timedelta(days=1)
    missing_after = generate_missing_dates(start_date, end_date, existing_data['date'])

    # Query missing data with regular saving (every 2500 entries)
    all_data_before, errors_before = [], []
    all_data_after, errors_after = [], []

    if not missing_before.empty:
        print(f"Querying missing dates before the CSV start")
        all_data_before, errors_before = query_data(
            missing_before, 
            query_number=1,
            save_frequency=2500,  # Save every 2500 entries
            csv_path=csv_path
        )

    if not missing_after.empty:
        print(f"Querying missing dates after the CSV end")
        all_data_after, errors_after = query_data(
            missing_after, 
            query_number=len(all_data_before) + 1,
            save_frequency=2500,  # Save every 2500 entries
            csv_path=csv_path
        )

    # Report statistics
    print(f"Total entries added: {len(all_data_before) + len(all_data_after)}")
    if errors_before or errors_after:
        print(f"Errors encountered on dates: {errors_before + errors_after}")

else:
    print("CSV file does not exist. Please create it before running the script.")

CSV file does not exist. Please create it before running the script.


In [10]:
### Code to check whether the CSV has changed strucutre

In [11]:
# import csv

# def check_csv_structure(filename, expected_fields_count=63):
#     with open(filename, 'r', newline='') as csvfile:
#         reader = csv.reader(csvfile)
#         header = next(reader)
#         if len(header) != expected_fields_count:
#             print(f"Header changed: Expected {expected_fields_count} fields but found {len(header)}.")
#         else:
#             print("Header matches expected field count.")
            
# # Replace 'data.csv' with your actual CSV file path
# check_csv_structure('csv_exports/bulk_results.csv')


In [12]:
# def find_incorrect_lines(filename, expected_fields_count=63):
#     bad_lines = []
#     with open(filename, 'r', newline='') as csvfile:
#         reader = csv.reader(csvfile)
#         for i, row in enumerate(reader, start=1):
#             if len(row) != expected_fields_count:
#                 bad_lines.append(i)
#     return bad_lines

# bad_lines = find_incorrect_lines('csv_exports/bulk_results.csv')
# print("Lines with incorrect field count:", bad_lines)


In [13]:
# import csv
# import linecache

# def export_context(filename, offending_line, context=10, output_file='offending_context.csv'):
#     # Read the header row
#     with open(filename, 'r', newline='') as f:
#         reader = csv.reader(f)
#         header = next(reader)
    
#     # Determine context range. We always include the header (line 1) separately.
#     # Since the header is line 1, our data lines start at line 2.
#     start_line = max(offending_line - context, 2)
#     end_line = offending_line + context
    
#     # Read the context lines using linecache.
#     lines = []
#     for i in range(start_line, end_line + 1):
#         line = linecache.getline(filename, i)
#         if line:
#             lines.append(line)
    
#     # Parse each context line using csv.reader
#     context_data = []
#     for l in lines:
#         # Parse each line; csv.reader returns an iterator so we grab the first row.
#         row = next(csv.reader([l]))
#         context_data.append(row)
    
#     # Determine the maximum column count between header and any context row.
#     max_columns = max(len(header), *(len(row) for row in context_data))
    
#     # If header is shorter than the maximum, extend it with empty strings.
#     if len(header) < max_columns:
#         header.extend([''] * (max_columns - len(header)))
    
#     # Also, pad each context row so they all have max_columns fields.
#     for row in context_data:
#         if len(row) < max_columns:
#             row.extend([''] * (max_columns - len(row)))
    
#     # Write the header and context rows to a new CSV file.
#     with open(output_file, 'w', newline='') as f_out:
#         writer = csv.writer(f_out)
#         writer.writerow(header)
#         writer.writerows(context_data)
    
#     print(f"Exported header and lines {start_line} to {end_line} to {output_file}.")

# # For example, to export context around the offending line 5109122:
# export_context('csv_exports/bulk_results.csv', offending_line=5109122, context=10)
